<a href="https://colab.research.google.com/github/soosysoda/fda_devices_tool/blob/main/text_extractor_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers sentence-transformers torch torchvision torchaudio PyMuPDF beautifulsoup4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 80.4 MB/s eta 0:00:00


In [6]:
import fitz  # PyMuPDF
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from transformers import pipeline
import torch
import re
from sentence_transformers import SentenceTransformer, util


In [7]:
device = 0 if torch.cuda.is_available() else -1
gpu_status = "GPU: " + torch.cuda.get_device_name(0) if device == 0 else "CPU"
print(f" Running on {gpu_status}")

 Running on GPU: Tesla T4


In [ ]:
from google.colab import drive
uploaded = files.upload()

Saving fda_bert_model.zip to fda_bert_model.zip


In [4]:
!unzip /content/drive/MyDrive/fda_bert_model.zip -d /content/drive/MyDrive/fda_bert_model

Archive:  /content/drive/MyDrive/fda_bert_model.zip
   creating: /content/drive/MyDrive/fda_bert_model/fda_bert_model/
  inflating: /content/drive/MyDrive/fda_bert_model/fda_bert_model/tokenizer_config.json  
  inflating: /content/drive/MyDrive/fda_bert_model/fda_bert_model/special_tokens_map.json  
  inflating: /content/drive/MyDrive/fda_bert_model/fda_bert_model/vocab.txt  
  inflating: /content/drive/MyDrive/fda_bert_model/fda_bert_model/config.json  
  inflating: /content/drive/MyDrive/fda_bert_model/fda_bert_model/model.safetensors  


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fda_bert_model/fda_bert_model")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/fda_bert_model/fda_bert_model")
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "/content/drive/MyDrive/fda_bert_model/fda_bert_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


id2label = {
    0: "Portable Scanner",
    1: "Rack Scanner",
    2: "Embedded Device",
    3: "Software Only",
    4: "Cloud Service",
    5: "Hybrid Cloud"
}
model.config.id2label = id2label
model.config.label2id = {v: k for k, v in id2label.items()}
print(model.config.id2label)

{0: 'Portable Scanner', 1: 'Rack Scanner', 2: 'Embedded Device', 3: 'Software Only', 4: 'Cloud Service', 5: 'Hybrid Cloud'}


In [11]:
from transformers import pipeline

clf_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)
embedder = SentenceTransformer("all-MiniLM-L6-v2")
print("Loaded model with", model.config.num_labels, "classes")
print("Label mapping:", model.config.id2label)

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded model with 6 classes
Label mapping: {0: 'Portable Scanner', 1: 'Rack Scanner', 2: 'Embedded Device', 3: 'Software Only', 4: 'Cloud Service', 5: 'Hybrid Cloud'}


In [12]:
CATEGORY_KEYWORDS = {
    "Hardware": ["probe", "sensor", "ultrasound", "scanner", "device", "unit", "hardware"],
    "Software Components": ["app", "module", "software", "system", "dashboard", "firmware"],
    "AI Models": ["ai", "ml", "classifier", "neural", "bert", "deep learning", "model"],
    "Data Pipelines": ["FHIR", "ETL", "stream", "cloud", "data ingestion", "API"],
    "User Interface": ["UI", "interface", "touchscreen", "display", "user input", "dashboard"],
    "Integration": ["EHR", "PACS", "HL7", "connect", "integration", "platform"]
}
CATEGORY_EMBEDDINGS = {
    category: embedder.encode(words, convert_to_tensor=True)
    for category, words in CATEGORY_KEYWORDS.items()
}

In [13]:
def extract_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join([page.get_text() for page in doc])

def get_all_internal_links(start_url, domain, max_pages=5, timeout=6):
    visited = set()
    to_visit = [start_url]
    found = []

    while to_visit and len(found) < max_pages:
        url = to_visit.pop(0)  # FIFO for breadth-first crawling
        if url in visited:
            continue
        visited.add(url)

        try:
            res = requests.get(url, timeout=timeout, headers={"User-Agent": "Mozilla/5.0"})
            if res.status_code != 200 or "text/html" not in res.headers.get("Content-Type", ""):
                continue

            soup = BeautifulSoup(res.text, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            text = re.sub(r"\s+", " ", text)
            found.append((url, text))

            for a in soup.find_all("a", href=True):
                full_url = urljoin(url, a["href"])
                if (
                    full_url.startswith(("http://", "https://"))
                    and urlparse(full_url).netloc == domain
                    and full_url not in visited
                ):
                    to_visit.append(full_url)

        except requests.RequestException:
            continue

    return found


def extract_design_with_bert(text):
    chunks = [text[i:i+1500] for i in range(0, len(text), 1500)]
    preds = []
    for chunk in chunks:
        result = clf_pipeline(chunk)[0]  # returns {"label": "...", "score": ...}
        preds.append(result["label"])
    return list(set(preds))

In [14]:
def normalize_keywords(raw_texts):
    bag = set()
    for raw in raw_texts:
        if not raw:
            continue
        parts = re.split(r"[,;\n]+", raw)
        for p in parts:
            kw = re.sub(r"\s+", " ", p).strip(" -–—_,.;:()[]").strip()
            if kw and 2 <= len(kw) < 120:
                bag.add(kw.lower())
    return sorted(bag)

def classify_keyword(keyword):
    kw_vec = embedder.encode(keyword, convert_to_tensor=True)
    max_sim = -1
    best_cat = "Uncategorized"
    for category, emb_list in CATEGORY_EMBEDDINGS.items():
        sim_scores = util.cos_sim(kw_vec, emb_list)
        top_score = sim_scores.max().item()
        if top_score > max_sim:
            max_sim = top_score
            best_cat = category
    return best_cat

def classify_keywords_bulk(keywords):
    result = {k: [] for k in CATEGORY_KEYWORDS}
    result["Uncategorized"] = []
    for kw in keywords:
        cat = classify_keyword(kw)
        result[cat].append(kw)
    return result

def format_to_markdown_table(classified):
    md = "| Category | Keywords (comma separated) |\n|----------|------------------------------|\n"
    for cat in list(CATEGORY_KEYWORDS.keys()) + ["Uncategorized"]:
        kws = ", ".join(sorted(set(classified.get(cat, []))))
        md += f"| {cat} | {kws} |\n"
    return md

In [15]:
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving K250005.pdf to K250005.pdf


In [16]:
print("\n Extracting from PDF...")
pdf_text = extract_pdf_text(pdf_path)
pdf_design_info = extract_design_with_bert(pdf_text)
for para in pdf_design_info:
    print(f" PDF: {para}")


 Extracting from PDF...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 PDF: Cloud Service
 PDF: Portable Scanner


In [17]:
pdf_keywords = normalize_keywords(pdf_design_info)

In [18]:
website_url = input("\n Enter official device website URL (or press Enter to skip): ").strip()
web_keywords = []
if website_url:
    domain = urlparse(website_url).netloc
    webpages = get_all_internal_links(website_url, domain, 10, 10)
    for url, content in webpages:
        design_info = extract_design_with_bert(content)
        for chunk in design_info:
            print(f" {url}: {chunk}")
        web_keywords += normalize_keywords(design_info)


 Enter official device website URL (or press Enter to skip): https://rivannamedical.com/
 https://rivannamedical.com/: Cloud Service
 https://rivannamedical.com/: Portable Scanner
 https://rivannamedical.com/why-accuro/: Cloud Service
 https://rivannamedical.com/why-accuro/: Portable Scanner
 https://rivannamedical.com/locator-kit/: Portable Scanner
 https://rivannamedical.com/locator-kit/: Hybrid Cloud
 https://rivannamedical.com/true-view/: Cloud Service
 https://rivannamedical.com/true-view/: Portable Scanner
 https://rivannamedical.com/true-view/: Hybrid Cloud
 https://rivannamedical.com/accuro-trolley/: Portable Scanner
 https://rivannamedical.com/accuro-resources/: Cloud Service
 https://rivannamedical.com/accuro-resources/: Portable Scanner
 https://rivannamedical.com/testimonials/: Cloud Service
 https://rivannamedical.com/testimonials/: Portable Scanner
 https://rivannamedical.com/clinical-value/: Cloud Service
 https://rivannamedical.com/clinical-value/: Portable Scanner
 ht

In [19]:
all_keywords = sorted(set(pdf_keywords + web_keywords))
print(all_keywords)

['cloud service', 'hybrid cloud', 'portable scanner']


In [20]:
classified = classify_keywords_bulk(all_keywords)
print("\n Classified Keywords Table:\n")
print(format_to_markdown_table(classified))


 Classified Keywords Table:

| Category | Keywords (comma separated) |
|----------|------------------------------|
| Hardware | portable scanner |
| Software Components |  |
| AI Models |  |
| Data Pipelines | cloud service, hybrid cloud |
| User Interface |  |
| Integration |  |
| Uncategorized |  |

